# ------ **Import Subroutines and Settings** -------

# Set up for importing data ie: **df_mf_ib77_nn** to add to the main data frame **df_mf_ib_tst**

In [1]:
import sys
print(sys.executable)
print("note: THIS IS THE DIRECTORY PYTHON IS WORKING IN.")

/home/bhuns/miniconda3/bin/python
note: THIS IS THE DIRECTORY PYTHON IS WORKING IN.


In [2]:
# Imports required for Loading, sorting .csx files to create specific data sets ie mrn inbody readings. 
%run ./sys_funcs.py              # loads all the def functions in sys_funcs.py into memory
#import sys_funcs                 # gives access to these def function digitalform that are in memory
from pathlib import Path
import csv
import pandas as pd
import numpy as np
import tkinter as tk
import pickle
from pathlib import Path
import csv
import os
import sys
from datetime import datetime
from datetime import time
from sys_funcs import read_csv_to_array
from sys_funcs import clean_wsl_path
from sys_funcs import array_to_dt_row_dict
from sys_funcs import make_blnk_update_row_dict
from sys_funcs import transpose_csv_to_col_dict
#from sys_funcs import update_values_with_config, get_update_result
from sys_funcs import transfer_updates
from sys_funcs import get_dtv_range
from sys_funcs import universal_import
from sys_funcs import parse_inbody_timestamp
from sys_funcs import build_lut
from sys_funcs import extract_a_column_as_df
from sys_funcs import extract_multicolumns_as_df
from sys_funcs import validate_and_sort_timestamps
from sys_funcs import extract_and_filter_by_time_window
from sys_funcs import read_file_dual_path
from sys_funcs import write_file_dual_path
from sys_funcs import asc_to_csv_cnv
from collections.abc import Mapping
import re
#from sys_funcs import 

In [3]:
# set print rows  This worksheet sets maximum # of rows printed
pd.set_option('display.max_rows', 1000)  # Adjust the number of rows to display
# pd.reset_option('display.max_rows')  
print('print set to 1000 rows max' )

print set to 1000 rows max


In [4]:
print("NOTE: timestamp = Test Date / Time does not work  use computed time stamp")

NOTE: timestamp = Test Date / Time does not work  use computed time stamp


# Pre-concatinate processing import and prepare 
1. **"df_m_ib_tst"**
2. **"df_m_ib77_nn"**


## def functions called in data importing & refinment.

In [51]:
media_lst = [
    "timestamp",
    "dtv",
    "ib_id",
    "cls",
    "cmmnts"
]

In [6]:
# 3rd version def drop_duplicates_by_test_time(df, keep='first', log=True):
def drop_duplicates_by_test_time(df, keep='first', log=True):
    """
    Removes duplicate rows based on the 'Test Date / Time' column.
    Keeps only the first (or last) occurrence.
    """
    df = df.copy()

    # Identify duplicate timestamps (beyond the one we keep)
    dupes = (
        df.loc[df.duplicated(subset=['Test Date / Time'], keep=keep), 'Test Date / Time']
        .astype(str)
        .values
        .tolist()
    )

    # Drop duplicate rows
    df = df.drop_duplicates(subset=['Test Date / Time'], keep=keep)

    # Optional logging
    if log and dupes:
        print("Removed duplicate rows for timestamps:", dupes)

    return df


In [7]:
# def strip_numbers_from_columns(df):
import re

def strip_numbers_from_columns(df):
    """
    Removes leading/trailing numbers and any leftover separators
    so that cases like '1.0ID' become 'ID'.
    """
    df = df.copy()
    new_cols = {}

    for col in df.columns:
        cleaned = col

        # Remove leading numbers + separators
        cleaned = re.sub(r'^\d+[\s\-\_\.:]*', '', cleaned)

        # Remove trailing numbers + separators
        cleaned = re.sub(r'[\s\-\_\.:]*\d+$', '', cleaned)

        # Remove leftover leading/trailing punctuation (.,-_:) after number removal
        cleaned = re.sub(r'^[\.\-\_\:]+', '', cleaned)
        cleaned = re.sub(r'[\.\-\_\:]+$', '', cleaned)

        new_cols[col] = cleaned

    return df.rename(columns=new_cols)


In [8]:
# Use drop duplicate function If duplicates are found
def drop_duplicate_columns(df, keep='first', log=True):
    """
    Removes duplicate column names from a DataFrame, keeping only the first
    (or last) occurrence. Useful after column-cleaning steps that may cause
    collisions. good i'm moving this around because I want to go ahead and do the I'm talking too

    Parameters
    ----------
    df : pandas.DataFrame
        Input DataFrame.
    keep : {'first', 'last'}, default 'first'
        Which duplicate to keep.
    log : bool, default True
        Whether to print which columns were removed.

    Returns
    -------
    pandas.DataFrame
        DataFrame with duplicate columns removed.
    """
    df = df.copy()

    # Identify duplicates beyond the one we keep
    dupes = df.columns[df.columns.duplicated(keep=keep)].tolist()

    # Drop them
    df = df.loc[:, ~df.columns.duplicated(keep=keep)]

    # Optional logging
    if log and dupes:
        print("Removed duplicate columns:", dupes)

    return df


In [9]:
# def prepend_empty_columns(df, col_list):
def prepend_empty_columns(df, col_list):
    """
    Prepend empty columns (from col_list) to the front of df.
    Returns a new DataFrame.
    """
    import pandas as pd

    # Create empty columns with same row count
    empty_df = pd.DataFrame(
        {col: [None] * len(df) for col in col_list}
    )

    # Prepend them
    return pd.concat([empty_df, df], axis=1)


In [10]:
# def fill_ib_media_cols(df):
#filling the media cols
def fill_ib_media_cols(df):
    """
    Fills the 5 leading operator columns for InBody datasets:
      - timestamp  ← parsed from 'Test Date / Time' (YYYYMMDDHHMMSS)
      - dtv        ← days since 1900‑01‑01
      - ib_id      ← 'mrn' if test time 03:00–23:59, else 'eve'
      - cls        ← NaN
      - cmmnts     ← NaN
    """

    df = df.copy()

    # --- 1. timestamp (correct parsing) ------------------------
    df['timestamp'] = pd.to_datetime(
        df['Test Date / Time'].astype(str),
        format="%Y%m%d%H%M%S",
        errors="coerce"
    )

    # --- 2. dtv: days since 1900‑01‑01 -------------------------
    origin = pd.Timestamp("1900-01-01")
    df['dtv'] = (df['timestamp'] - origin).dt.days

    # --- 3. ib_id classification -------------------------------
    def classify_ib_id(ts):
        if pd.isna(ts):
            return np.nan
        hour = ts.hour
        return "mrn" if 3 <= hour <= 12 else "eve"

    df['ib_id'] = df['timestamp'].apply(classify_ib_id)

    # --- 4. cls ------------------------------------------------
    df['cls'] = np.nan

    # --- 5. cmmnts ---------------------------------------------
    df['cmmnts'] = np.nan

    return df


In [11]:
# Sort the rows by timestamp
def sort_by_timestamp(df):
    """
    Sorts an InBody dataframe by the 'timestamp' column
    in ascending chronological order.
    """
    df = df.copy()
    df = df.sort_values(by='timestamp', ascending=True)
    df = df.reset_index(drop=True)
    return df


In [12]:
# remove duplicates on the basis of timestamp
def remove_ib_duplicates(df, subset_cols=None):
    """
    Removes duplicate InBody rows based on key identifying columns.
    Default behavior: remove duplicates based on ['ID', 'timestamp'].
    """
    df = df.copy()

    # Default duplicate definition
    if subset_cols is None:
        subset_cols = ['timestamp']
        # subset_cols = ['ID', 'timestamp']
    # Remove duplicates, keeping the first occurrence
    df = df.drop_duplicates(subset=subset_cols, keep='first')

    # Reset index for cleanliness
    df = df.reset_index(drop=True)

    return df


In [135]:
# veirfy if rows exixt in master_timestamps(df_master, df_new, ts_col="timestamp"):
def filter_new_rows_by_master_timestamps(df_master, df_new, ts_col="timestamp"):
    """
    Filters df_new so that only rows whose timestamps appear in df_master remain.

    Parameters
    ----------
    df_master : pd.DataFrame
        The master dataframe containing valid timestamps.
    df_new : pd.DataFrame
        The new dataframe to be filtered.
    ts_col : str, optional
        The name of the timestamp column (default is 'timestamp').

    Returns
    -------
    pd.DataFrame
        A filtered version of df_new containing only rows whose timestamps
        exist in df_master.
    """

    # Extract the set of valid timestamps from the master dataframe
    valid_timestamps = set(df_master[ts_col])

    # Filter df_new to keep only rows with timestamps in the master set
    df_filtered = df_new[df_new[ts_col].isin(valid_timestamps)].copy()

    return df_filtered


In [134]:
# def keep_only_new_timestamps(df_master, df_new, ts_col="timestamp")
def keep_only_new_timestamps(df_master, df_new, ts_col="timestamp"):
    """
    Returns only the rows in df_new whose timestamps do NOT exist in df_master.

    Parameters
    ----------
    df_master : pd.DataFrame
        The master dataframe containing timestamps already ingested.
    df_new : pd.DataFrame
        The new dataframe to be filtered.
    ts_col : str, optional
        The name of the timestamp column (default is 'timestamp').

    Returns
    -------
    pd.DataFrame
        A filtered version of df_new containing only rows with timestamps
        NOT present in df_master.
    """

    # Extract the set of timestamps already in the master
    existing_ts = set(df_master[ts_col])

    # Keep only rows whose timestamp is NOT in the master
    df_filtered = df_new[~df_new[ts_col].isin(existing_ts)].copy()

    return df_filtered


In [137]:
# def append_rows_with_master_schema(master_df, adder_df):
import pandas as pd
import numpy as np

def append_rows_with_master_schema(master_df, adder_df):
    """
    Appends rows from adder_df into master_df while enforcing the master_df schema.

    For each row in adder_df:
      - Columns that exist in adder_df are copied.
      - Columns missing from adder_df are filled with NaN.
      - All master_df columns are preserved in order.

    Parameters
    ----------
    master_df : pd.DataFrame
        The master dataframe with the full schema.
    adder_df : pd.DataFrame
        The dataframe containing rows to append (subset of master columns).

    Returns
    -------
    pd.DataFrame
        Updated master_df with new rows appended.
    """

    # Reindex adder_df to match master_df columns, filling missing columns with NaN
    adder_aligned = adder_df.reindex(columns=master_df.columns)

    # Append and return
    return pd.concat([master_df, adder_aligned], ignore_index=True)


# UPDATE **"df_mf_ib_tst"** with latest data

## Bringing in the main data file from Excel and produce clean **df_mf_ib_tst_nn** 

### load from XL

In [128]:
#this brings in the main data file from Excel
df_mf_ib_tst = pd.read_csv("/home/bhuns/JL_2/data/ib_tst/mf_ib_tst.csv")
df_mf_ib_tst_nn = strip_numbers_from_columns(df_m_ib_tst)
# verify 
df_mf_ib_tst_nn
#print("imported **df_m_ib_tst_nn** ie numbers removed from col heads")

,dtv,timestamp,cls,ib_id,cmmnts,Name,ID,Height,Date of Birth,Age,...,Weight_Z score,kHz-Whole Body Phase Angle_T score,kHz-Whole Body Phase Angle_Z score,TBW/WT_T Score,TBW/WT_Z Score,SMI(SMM/Wt)_T score,SMI(SMM/Wt)_Z score,ECM/BCM_T Score,ECM/BCM Z Score,Unnamed
0,NaN,NaN,NaN,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Insure no numbers in col names & slice-abilty

In [105]:
# df_m_ib_tst_nn slicing
# verify 
print('df_m_ib_tst_nn imported \n Test Date / Time \n',df_m_ib_tst_nn['Test Date / Time'])  # list form wo meta


df_m_ib_tst_nn imported 
 Test Date / Time 
 0     20251213054921
1     20251214234814
2     20251217075000
3     20251218081604
4     20251216233809
5     20251215080038
6     20251212221713
7     20251219080729
8     20251214090336
9     20251218212747
10    20251219232758
11    20251215233324
12    20251216064808
13    20251211092610
14    20251217230745
15    20251220085024
16    20251212084231
17    20251213054921
18    20251214234814
19    20251217075000
20    20251218081604
21    20251216233809
22    20251215080038
23    20251212221713
24    20251219080729
25    20251214090336
26    20251218212747
27    20251219232758
28    20251215233324
29    20251216064808
30    20251211092610
31    20251217230745
32    20251220085024
33    20251212084231
34    20251213054921
35    20251214234814
36    20251217075000
37    20251218081604
38    20251216233809
39    20251215080038
40    20251212221713
41    20251219080729
42    20251214090336
43    20251218212747
44    20251219232758
45    2025

### Insure that media_lst cols inserted in **df_m_ib_tst_nn**

In [106]:
# verify 
df_m_ib_tst_nn[media_lst]

,timestamp,dtv,ib_id,cls,cmmnts
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN


### Insure that media_lst cols are filled in **df_mf_ib_tst_nn**

In [107]:
# fill_ib_media_cols(df_mf_ib_tst_nn)
df_mf_ib_tst_nn = fill_ib_media_cols(df_m_ib_tst_nn)
# verify 
df_mf_ib_tst_nn[media_lst]

,timestamp,dtv,ib_id,cls,cmmnts
0,2025-12-13 05:49:21,46002,mrn,NaN,NaN
1,2025-12-14 23:48:14,46003,eve,NaN,NaN
2,2025-12-17 07:50:00,46006,mrn,NaN,NaN
3,2025-12-18 08:16:04,46007,mrn,NaN,NaN
4,2025-12-16 23:38:09,46005,eve,NaN,NaN
5,2025-12-15 08:00:38,46004,mrn,NaN,NaN
6,2025-12-12 22:17:13,46001,eve,NaN,NaN
7,2025-12-19 08:07:29,46008,mrn,NaN,NaN
8,2025-12-14 09:03:36,46003,mrn,NaN,NaN
9,2025-12-18 21:27:47,46007,eve,NaN,NaN


## Import new data from ip77 in the data folder of repo and produce clean rows ready to concatinate

### This segment imports the data from the Excel file probo/data/ib77 to dataframe with numbers in col names

In [108]:
# loads the new from the 770
df_ib77_raw = universal_import(
    folder_path="/home/bhuns/JL_2/data/ib77",
    pattern="*"
)
# verify df_ib77_raw #print("df_ib77_raw w/o numbers OK")

✅ Loaded 091725_20251225081906.csv with utf-8
✅ Loaded 091725_20251213054921.csv with utf-8
✅ Loaded 091725_20251214234814.csv with utf-8
✅ Loaded 091725_20251229081757.csv with utf-8
✅ Loaded 091725_20251223231059.csv with utf-8
✅ Loaded 091725_20251217075000.csv with utf-8
✅ Loaded 091725_20251218081604.csv with utf-8
✅ Loaded 091725_20251221071026.csv with utf-8
✅ Loaded 091725_20251216233809.csv with utf-8
✅ Loaded 091725_20251215080038.csv with utf-8
✅ Loaded 091725-1_20251212221713.csv with utf-8
✅ Loaded 091725_20251219080729.csv with utf-8
✅ Loaded 091725_20251221205127.csv with utf-8
✅ Loaded 091725_20251226083412.csv with utf-8
✅ Loaded 091725_20251227083952.csv with utf-8
✅ Loaded 091725_20251214090336.csv with utf-8
✅ Loaded 091725_20251218212747.csv with utf-8
✅ Loaded 091725_20251228225544.csv with utf-8
✅ Loaded 091725_20251224080342.csv with utf-8
✅ Loaded 091725_20251222221501.csv with utf-8
✅ Loaded 091725_20251219232758.csv with utf-8
✅ Loaded 091725_20251215233324.c

### This segment strips off the col names of numbers and produces "df_m_ib77_nn" and demonstrates slicing

In [109]:
# verify slicing 
df_ib77_nn[['ID','Test Date / Time','ECW/TBW']]  #,"SMM (Skeletal Muscle Mass)" , "Weight" , "BMR (Basal Metabolic Rate)" , "ECW/TBW"]].head(20)

,ID,Test Date / Time,ECW/TBW
0,6ft 06.0in,20251225081906,0.399
1,6ft 06.0in,20251213054921,0.410
2,6ft 06.0in,20251214234814,0.414
3,6ft 06.0in,20251229081757,0.398
4,6ft 06.0in,20251223231059,0.405
5,6ft 06.0in,20251217075000,0.404
6,6ft 06.0in,20251218081604,0.403
7,6ft 06.0in,20251221071026,0.400
8,6ft 06.0in,20251216233809,0.411
9,6ft 06.0in,20251215080038,0.406


In [110]:
df_ib77_nn = strip_numbers_from_columns(df_ib77_raw)
#verify 
print(list(df_ib77_nn.columns))

['Name', 'ID', 'Height', 'Date of Birth', 'Age', 'Mobile Number', 'Phone Number', 'Zip Code', 'Address', 'E-mail', 'Date of Registration', 'Memo', 'Test Date / Time', 'Weight', 'TBW (Total Body Water)', 'ICW (Intracellular Water)', 'ECW (Extracellular Water)', 'DLM (Dry Lean Mass)', 'BFM (Body Fat Mass)', 'LBM (Lean Body Mass)', 'SMM (Skeletal Muscle Mass)', 'BMI (Body Mass Index)', 'PBF (Percent Body Fat)', 'LBM of Right Arm', 'LBM% of Right Arm', 'LBM of Left Arm', 'LBM% of Left Arm', 'LBM of Trunk', 'LBM% of Trunk', 'LBM of Right Leg', 'LBM% of Right Leg', 'LBM of Left Leg', 'LBM% of Left Leg', 'Leg Lean Mass', 'TBW of Right Arm', 'TBW of Left Arm', 'TBW of Trunk', 'TBW of Right Leg', 'TBW of Left Leg', 'ICW of Right Arm', 'ICW of Left Arm', 'ICW of Trunk', 'ICW of Right Leg', 'ICW of Left Leg', 'ECW of Right Arm', 'ECW of Left Arm', 'ECW of Trunk', 'ECW of Right Leg', 'ECW of Left Leg', 'ECW/TBW', 'ECW/TBW of Right Arm', 'ECW/TBW of Left Arm', 'ECW/TBW of Trunk', 'ECW/TBW of Right 

### This segment eliminates duplicates in **df_ib77_nn**

In [111]:
# Seek out and drop duplicates
df_ib77_nn = drop_duplicate_columns(df_ib77_nn)

Removed duplicate columns: ['Systolic', 'Diastolic', 'Pulse', 'Mean Artery Pressure', 'Pulse Pressure', 'Rate Pressure Product']


In [112]:
# Testing duplicates in the column titles
df_ib77_nn.columns[df_ib77_nn.columns.duplicated()]

print ("df_ib77_nn , No numbers, no duplicates OK")

df_ib77_nn , No numbers, no duplicates OK


### This segment adds **media_cols** and fills them from data in the results **"df_m_ib77_nn"**

In [113]:
df_m_ib77_nn = prepend_empty_columns(df_ib77_nn, media_lst)
# verify 
df_m_ib77_nn

,timestamp,dtv,ib_id,cls,cmmnts,Name,ID,Height,Date of Birth,Age,...,R/Ht,Xc/Ht,HGS of Left Arm 1st,HGS of Left Arm 2nd,HGS of Right Arm 1st,HGS of Right Arm 2nd,HGS/WT,Unnamed,source_file,encoding_used
0,None,None,None,None,None,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251225081906.csv,utf-8
1,None,None,None,None,None,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251213054921.csv,utf-8
2,None,None,None,None,None,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251214234814.csv,utf-8
3,None,None,None,None,None,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251229081757.csv,utf-8
4,None,None,None,None,None,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251223231059.csv,utf-8
5,None,None,None,None,None,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251217075000.csv,utf-8
6,None,None,None,None,None,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251218081604.csv,utf-8
7,None,None,None,None,None,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251221071026.csv,utf-8
8,None,None,None,None,None,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251216233809.csv,utf-8
9,None,None,None,None,None,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251215080038.csv,utf-8


### This segment eliminates duplicates in **df_m_ib77_nn**

In [114]:
df_m_ib77_nn = drop_duplicate_columns(df_m_ib77_nn)
print ("df_m_ib77_nn , No numbers, no duplicates OK")

df_m_ib77_nn , No numbers, no duplicates OK


In [115]:
# verify 
df_m_ib77_nn[media_lst]

,timestamp,dtv,ib_id,cls,cmmnts
0,None,None,None,None,None
1,None,None,None,None,None
2,None,None,None,None,None
3,None,None,None,None,None
4,None,None,None,None,None
5,None,None,None,None,None
6,None,None,None,None,None
7,None,None,None,None,None
8,None,None,None,None,None
9,None,None,None,None,None


In [116]:
df_m_ib77_nn = drop_duplicate_columns(df_m_ib77_nn)


In [117]:
df_m_ib77_nn.columns[df_m_ib77_nn.columns.duplicated()]
print("No duplicates detected")

No duplicates detected


In [118]:
df_m_ib77_nn['Test Date / Time']

0     20251225081906
1     20251213054921
2     20251214234814
3     20251229081757
4     20251223231059
5     20251217075000
6     20251218081604
7     20251221071026
8     20251216233809
9     20251215080038
10    20251212221713
11    20251219080729
12    20251221205127
13    20251226083412
14    20251227083952
15    20251214090336
16    20251218212747
17    20251228225544
18    20251224080342
19    20251222221501
20    20251219232758
21    20251215233324
22    20251224234318
23    20251216064808
24    20251211092610
25    20251223082717
26    20251217230745
27    20251226232556
28    20251220085024
29    20251222083404
30    20251228081055
31    20251212084231
32    20251226004351
33    20251220233943
Name: Test Date / Time, dtype: int64

In [123]:
# fill_ib_media_cols(df_m_ib77_nn)
df_mf_ib77_nn = fill_ib_media_cols(df_m_ib77_nn)
# verify 
df_mf_ib77_nn

,timestamp,dtv,ib_id,cls,cmmnts,Name,ID,Height,Date of Birth,Age,...,R/Ht,Xc/Ht,HGS of Left Arm 1st,HGS of Left Arm 2nd,HGS of Right Arm 1st,HGS of Right Arm 2nd,HGS/WT,Unnamed,source_file,encoding_used
0,2025-12-25 08:19:06,46014,mrn,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251225081906.csv,utf-8
1,2025-12-13 05:49:21,46002,mrn,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251213054921.csv,utf-8
2,2025-12-14 23:48:14,46003,eve,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251214234814.csv,utf-8
3,2025-12-29 08:17:57,46018,mrn,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251229081757.csv,utf-8
4,2025-12-23 23:10:59,46012,eve,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251223231059.csv,utf-8
5,2025-12-17 07:50:00,46006,mrn,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251217075000.csv,utf-8
6,2025-12-18 08:16:04,46007,mrn,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251218081604.csv,utf-8
7,2025-12-21 07:10:26,46010,mrn,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251221071026.csv,utf-8
8,2025-12-16 23:38:09,46005,eve,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251216233809.csv,utf-8
9,2025-12-15 08:00:38,46004,mrn,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251215080038.csv,utf-8


### This segment eliminates duplicates based on **"test_time" in **"df_mf_ib77_nn"**

In [124]:
df_mf_ib77_nn.columns[df_mf_ib77_nn.columns.duplicated()]
print("No duplicates **df_mf_ib77_nn** detected")

No duplicates **df_mf_ib77_nn** detected


In [125]:
# verify 
df_mf_ib_tst_nn[media_lst]

,timestamp,dtv,ib_id,cls,cmmnts
0,2025-12-13 05:49:21,46002,mrn,NaN,NaN
1,2025-12-14 23:48:14,46003,eve,NaN,NaN
2,2025-12-17 07:50:00,46006,mrn,NaN,NaN
3,2025-12-18 08:16:04,46007,mrn,NaN,NaN
4,2025-12-16 23:38:09,46005,eve,NaN,NaN
5,2025-12-15 08:00:38,46004,mrn,NaN,NaN
6,2025-12-12 22:17:13,46001,eve,NaN,NaN
7,2025-12-19 08:07:29,46008,mrn,NaN,NaN
8,2025-12-14 09:03:36,46003,mrn,NaN,NaN
9,2025-12-18 21:27:47,46007,eve,NaN,NaN


### This segment sorts based on **"timestamp"** in **"df_mf_ib77_nn"** with results **df_mf_ib77_nn_s**

In [126]:
#sort_by_timestamp(df_mf_ib77_nn)
df_mf_ib77_nn_s = sort_by_timestamp(df_mf_ib77_nn)
#verify 
df_mf_ib77_nn_s

,timestamp,dtv,ib_id,cls,cmmnts,Name,ID,Height,Date of Birth,Age,...,R/Ht,Xc/Ht,HGS of Left Arm 1st,HGS of Left Arm 2nd,HGS of Right Arm 1st,HGS of Right Arm 2nd,HGS/WT,Unnamed,source_file,encoding_used
0,2025-12-11 09:26:10,46000,mrn,NaN,NaN,251211-1,6ft 06.0in,1939.02.23.,M,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725-1_20251211092610.csv,utf-8
1,2025-12-12 08:42:31,46001,mrn,NaN,NaN,251211-1,6ft 06.0in,1939.02.23.,M,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725-1_20251212084231.csv,utf-8
2,2025-12-12 22:17:13,46001,eve,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725-1_20251212221713.csv,utf-8
3,2025-12-13 05:49:21,46002,mrn,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251213054921.csv,utf-8
4,2025-12-14 09:03:36,46003,mrn,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251214090336.csv,utf-8
5,2025-12-14 23:48:14,46003,eve,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251214234814.csv,utf-8
6,2025-12-15 08:00:38,46004,mrn,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251215080038.csv,utf-8
7,2025-12-15 23:33:24,46004,eve,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251215233324.csv,utf-8
8,2025-12-16 06:48:08,46005,mrn,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251216064808.csv,utf-8
9,2025-12-16 23:38:09,46005,eve,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251216233809.csv,utf-8


## Eliminate all rows that have time stamps equal to a timestamp in the master data frame: **df_mf_ib_tst** yielding **df_mf_ib77_nn_sn** ....*"sn"* means *"sorted new"*

# WORKING cell 
1. Verify this Timestamp has not been recorded
2. If not insert value col by col

In [ ]:
# Verify that we have the main data frame
df_m_ib_tst.columns

In [ ]:
#Verify that we have data from data frame to be added to the main data frame 
# If it's not there read it from pickle
# verify df_ib77_raw.columns

In [ ]:
# READ the IB77 Columns without numbers FROM pickle 
with open("df_ib77_raw_nms.pkl", "rb") as f:
    df_ib77_raw_nms = pickle.load(f)
# Verify df_ib77_raw_nms
print ("from pkl df_ib77_raw_nms.pkl OK")

In [ ]:
# Load the IB77 Columns without numbers to pickle 
with open("df_ib77_raw_nms.pkl", "wb") as f:
    pickle.dump(df_ib77_raw_nms, f)
print ("df_ib77_raw_nms.pkl OK")

In [136]:
df_mf_ib77_nn_sn = keep_only_new_timestamps(
    df_mf_ib_tst_nn,
    df_mf_ib77_nn_s,
    ts_col="timestamp"
)
# verify 
df_mf_ib77_nn_sn

,timestamp,dtv,ib_id,cls,cmmnts,Name,ID,Height,Date of Birth,Age,...,R/Ht,Xc/Ht,HGS of Left Arm 1st,HGS of Left Arm 2nd,HGS of Right Arm 1st,HGS of Right Arm 2nd,HGS/WT,Unnamed,source_file,encoding_used
0,2025-12-11 09:26:10,46000,mrn,NaN,NaN,251211-1,6ft 06.0in,1939.02.23.,M,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725-1_20251211092610.csv,utf-8
1,2025-12-12 08:42:31,46001,mrn,NaN,NaN,251211-1,6ft 06.0in,1939.02.23.,M,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725-1_20251212084231.csv,utf-8
2,2025-12-12 22:17:13,46001,eve,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725-1_20251212221713.csv,utf-8
3,2025-12-13 05:49:21,46002,mrn,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251213054921.csv,utf-8
4,2025-12-14 09:03:36,46003,mrn,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251214090336.csv,utf-8
5,2025-12-14 23:48:14,46003,eve,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251214234814.csv,utf-8
6,2025-12-15 08:00:38,46004,mrn,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251215080038.csv,utf-8
7,2025-12-15 23:33:24,46004,eve,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251215233324.csv,utf-8
8,2025-12-16 06:48:08,46005,mrn,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251216064808.csv,utf-8
9,2025-12-16 23:38:09,46005,eve,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251216233809.csv,utf-8


In [138]:
df_mf_ib_tst = append_rows_with_master_schema(
    df_mf_ib_tst,
    df_mf_ib77_nn_sn
)
# verify 
df_mf_ib_tst


/tmp/ipykernel_978164/4064424017.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([master_df, adder_aligned], ignore_index=True)


,dtv,timestamp,cls,ib_id,cmmnts,Name,ID,Height,Date of Birth,Age,...,Weight_Z score,50kHz-Whole Body Phase Angle_T score,50kHz-Whole Body Phase Angle_Z score,TBW/WT_T Score,TBW/WT_Z Score,SMI(SMM/Wt)_T score,SMI(SMM/Wt)_Z score,ECM/BCM_T Score,ECM/BCM Z Score,Unnamed: 250
0,NaN,NaT,NaN,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaT,NaN,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaT,NaN,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaT,NaN,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaT,NaN,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaT,NaN,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaT,NaN,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaT,NaN,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaT,NaN,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaT,NaN,NaN,NaN,91725,6ft 06.0in,1939.02.23.,F,86.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
